In [25]:
%load_ext autoreload
%autoreload 2
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from gensim.models import Word2Vec
from collections import defaultdict
from scipy.spatial.distance import cosine as cosine_distance
import random
random.seed(5)
import json
import sys
import random
sys.path.append('../src')
from models import word_set_to_mtx, save_pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
we_model_name = "sg_dim300_min100_win5"
we_vector_size = 300
we_model_dir = '../data/external/wiki-english/wiki-english-20171001/%s' % we_model_name

we_model = Word2Vec.load(we_model_dir+'/model.gensim')
print ('loading done!')
print(f'Total words: {len(we_model.wv.vocab)}')

loading done!
Total words: 312425


In [22]:
THRESHOLD_BIASES_PATH_2NDORDER = '../data/processed/threshold_biases_2ndorder.pickle'
THRESHOLD_BIASES_PATH_1STORDER = '../data/processed/threshold_biases_1storder.pickle'

In [3]:
vocabulary = list(we_model.wv.vocab.keys())
random.choices(vocabulary, k=3)

['wftv', 'stollwerck', 'dansville']

In [4]:
set_1 = random.choices(vocabulary, k=3)
set_2 = random.choices(vocabulary, k=3)

In [5]:
set_1 = np.array([[1, 1, 1], [2,5,10], [3,5,10]])
set_2 = np.array([[1, 1, 1], [5, 5, 5], [10,10,10]])
zip(set_1, set_2)
[print(u,v) for u,v in zip(set_1, set_2)]

[1 1 1] [1 1 1]
[ 2  5 10] [5 5 5]
[ 3  5 10] [10 10 10]


[None, None, None]

In [14]:
def matrix_cosine(x, y):
    return np.einsum('ij,ij->i', x, y) / (
              np.linalg.norm(x, axis=1) * np.linalg.norm(y, axis=1)
    )
print(type(set_1))
matrix_cosine(set_1, set_2)

<class 'numpy.ndarray'>


array([1.        , 0.86415857, 0.8977584 ])

In [28]:
def get_biases(we_model, k_pairs=10000, order='second'):
    vocabulary = list(we_model.wv.vocab.keys())
    set_1 = random.choices(vocabulary, k=k_pairs)
    set_2 = random.choices(vocabulary, k=k_pairs)
    arr_1 = word_set_to_mtx(we_model, set_1)
    arr_2 = word_set_to_mtx(we_model, set_2)
    if order == 'second':
        biases = matrix_cosine(arr_1, arr_2)
        save_pickle(biases, THRESHOLD_BIASES_PATH_2NDORDER)
    else:
        # Insert first-order function here, then remove "break" line
        save_pickle(biases, THRESHOLD_BIASES_PATH_1STORDER)
    return biases
get_biases(we_model)

array([0.16863719, 0.12766281, 0.07611948, ..., 0.06586456, 0.19470786,
       0.03731716], dtype=float32)